# Simple Image Analysis LLM

Run Llama3.2 Vision with Langchain and Ollama

In [12]:
! pip install -Uqqq pip --progress-bar off
! pip install -qqq transformers -U --progress-bar off
! pip install sentence-transformers  langchain langchain-community langchain-huggingface trl datasets pypdf  -qqq --progress-bar off
! pip install torch torchvision -qqq --progress-bar off
! pip install langchain-ollama "ollama==0.4.2" -q

ERROR: To modify pip, please run the following command:
C:\Program Files\Python311\python.exe -m pip install -Uqqq pip --progress-bar off

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from langchain_ollama import OllamaEmbeddings

In [17]:
import base64
from io import BytesIO

from  IPython.display import display
from PIL import Image


def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Re-sized Base64 string
    """

    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str


def plt_img_base64(img_base64):
    """
    Disply base64 encoded string as image

    :param img_base64:  Base64 string
    """
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))


file_path = "job_requirements.jpg"
pil_image = Image.open(file_path)
image_b64 = convert_to_base64(pil_image)

In [20]:
from langchain_core.messages import HumanMessage
from langchain_ollama import  ChatOllama
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model="llama3.2-vision", temperature=0)

def prompt_func(data):
    text = data["text"]
    image = data["image"]

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }

    content_parts = []

    text_part = {"type": "text", "text": text}

    content_parts.append(image_part)
    content_parts.append(text_part)

    return [HumanMessage(content=content_parts)]


chain = prompt_func | llm | StrOutputParser()

query_chain = chain.invoke(
    {"text": "Does this resume has a requirement for Docker ?", "image": image_b64}
)

query_chain

'**Yes**, the resume requires experience with **Docker components**. This is mentioned in the Responsibilities section, which states that the candidate should be able to develop and interact with computer vision models, including developing Docker components to be consumed by workflow-based applications.'